In [1]:
import sys
import PyEcoNets as pen
import pandas as pd
import os

WORK_DIR = os.getcwd()

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import odeint
from matplotlib import rcParams

In [27]:
#SSP2
# Function to run the initial simulation
def run_initial_simulation(network_id, model,  ini_folder, results_folder):
    ini_path = os.path.join(ini_folder, f"{model}_{network_id}_SSP2.ini")
    config = pen.GlobalConfig(ini_path)
    Sampler = pen.SimSpecsSampler(config.params)
    runner = pen.Simulator(config, Sampler.AllSimSpecs[-1])
    
    results_filename = os.path.join(results_folder, f"initial_pop_{network_id}.csv")
    
    SpeciesPopEnd, Data = runner.run(
        IsManaged=False,
        IsTemperatureDependent=None,
        SpeciesPopEnd=None,
        t_i=None,
        t_f=None,
        t_steps=None,
        show_progress=True,
        show_plot=False,
        ReturnPopEnd=True,
        save_results=True,
        results_filename=results_filename,
        ReturnData=True
    )
    
    return SpeciesPopEnd, Data

# Function to run the subsequent simulation for the 1850-2014 period
def run_subsequent_simulation_1850_2014(network_id, model, initial_pop_end, ini_folder, results_folder):
    ini_path = os.path.join(ini_folder, f"{model}_{network_id}_SSP2.ini")
    config = pen.GlobalConfig(ini_path)
    Sampler = pen.SimSpecsSampler(config.params)

    for i in range(len(Sampler.AllSimSpecs)):
        runner = pen.Simulator(config, Sampler.AllSimSpecs[i])
        results_filename = os.path.join(results_folder, f"without_{model}_SSP2_pop_{network_id}_1850_2014{str(runner.sim_specs['gamma0']).replace('.', 'p')}.csv")
        
        _, _ = runner.run(
            IsManaged=None,
            IsTemperatureDependent=True,
            SpeciesPopEnd=initial_pop_end,
            t_i=None,
            t_f=None,
            t_steps=None,
            show_progress=True,
            show_plot=False,
            ReturnPopEnd=True,
            save_results=True,
            results_filename=results_filename,
            ReturnData=True
        )

# Function to run the subsequent simulation for the 2015-2100 period
def run_subsequent_simulation_2015_2100(network_id, model, ini_folder, initial_folder, results_folder):
    ini_path = os.path.join(ini_folder, f"{model}_{network_id}_SSP5.ini")
    config = pen.GlobalConfig(ini_path)
    Sampler = pen.SimSpecsSampler(config.params)

    for i in range(len(Sampler.AllSimSpecs)):
        temp_csv = os.path.join(initial_folder, f"without_{model}_SSP2_pop_{network_id}_1850_2014{str(Sampler.AllSimSpecs[i]['gamma0']).replace('.', 'p')}.csv")
        SpeciesInitial = pd.read_csv(temp_csv).iloc[-1]
        SpeciesInitial.drop(['Unnamed: 0', 'year', 'month', 'date'], inplace=True)
        
        runner = pen.Simulator(config, Sampler.AllSimSpecs[i])
        results_filename = os.path.join(results_folder, f"without_{model}_SSP2_pop_{network_id}_{str(runner.sim_specs['gamma0']).replace('.', 'p')}.csv")
        
        _, Data = runner.run(
            IsManaged=None,
            IsTemperatureDependent=True,
            SpeciesPopEnd=SpeciesInitial,
            t_i=None,
            t_f=None,
            t_steps=None,
            show_progress=True,
            show_plot=False,
            ReturnPopEnd=True,
            save_results=True,
            results_filename=results_filename,
            ReturnData=True
        )

# Main function to execute the simulations
def main():
    csv_file = f'{WORK_DIR}/Network_location/Network_location.csv'
    networks_df = pd.read_csv(csv_file)

    models = ['BCC','CESM2','EC','AWI', 'FGOALS', 'MRI', 'INM', 'CMCC', 'NorESM2', 'EC_Veg']

    for model in models:

        ini_folder = f'{WORK_DIR}/ini_files/'
        ini_1850_2014 = f'{WORK_DIR}/1850_2014_ini_files/'
        ini_2015_2100 = f'{WORK_DIR}/2015_2100_ini_files/'
        initial_folder = f'{WORK_DIR}/Results/1850_2014/SSP2_{model}'
        results_folder_1850_2014 = initial_folder
        results_folder_2015_2100 = f"{WORK_DIR}/Results/2015_2100/without/SSP2_{model}"

        os.makedirs(results_folder_1850_2014, exist_ok=True)
        os.makedirs(results_folder_2015_2100, exist_ok=True)

        for _, row in networks_df.iterrows():
            network_id = row['Network ID']

            # Run initial simulation
            SpeciesPopEnd, _ = run_initial_simulation(network_id, model, ini_folder, results_folder_1850_2014)

            # Run subsequent simulation for 1850-2014
            run_subsequent_simulation_1850_2014(network_id, model, SpeciesPopEnd, ini_1850_2014, results_folder_1850_2014)

            # Run subsequent simulation for 2015-2100
            run_subsequent_simulation_2015_2100(network_id, model, ini_2015_2100, initial_folder, results_folder_2015_2100)

    print("Simulations completed for all models and scenarios.")

if __name__ == "__main__":
    main()


In [ ]:
#SSP5
# Function to run the subsequent simulation for the 2015-2100 period
def run_subsequent_simulation_2015_2100(network_id, model, ini_folder, initial_folder, results_folder):
    ini_path = os.path.join(ini_folder, f"{model}_{network_id}_SSP5.ini")
    config = pen.GlobalConfig(ini_path)
    Sampler = pen.SimSpecsSampler(config.params)

    for i in range(len(Sampler.AllSimSpecs)):
        temp_csv = os.path.join(initial_folder, f"without_{model}_SSP2_pop_{network_id}_1850_2014{str(Sampler.AllSimSpecs[i]['gamma0']).replace('.', 'p')}.csv")
        SpeciesInitial = pd.read_csv(temp_csv).iloc[-1]
        SpeciesInitial.drop(['Unnamed: 0', 'year', 'month', 'date'], inplace=True)
        
        runner = pen.Simulator(config, Sampler.AllSimSpecs[i])
        results_filename = os.path.join(results_folder, f"without_{model}_SSP5_pop_{network_id}_{str(runner.sim_specs['gamma0']).replace('.', 'p')}.csv")
        
        _, Data = runner.run(
            IsManaged=None,
            IsTemperatureDependent=True,
            SpeciesPopEnd=SpeciesInitial,
            t_i=None,
            t_f=None,
            t_steps=None,
            show_progress=True,
            show_plot=False,
            ReturnPopEnd=True,
            save_results=True,
            results_filename=results_filename,
            ReturnData=True
        )

# Main function to execute the simulations
def main():
    csv_file = f'{WORK_DIR}/Network_location/Network_location.csv'
    networks_df = pd.read_csv(csv_file)

    models = ['BCC','CESM2','EC','AWI', 'FGOALS', 'MRI', 'INM', 'CMCC', 'NorESM2', 'EC_Veg']

    for model in models:

        ini_folder = f'{WORK_DIR}/ini_files/'
        ini_1850_2014 = f'{WORK_DIR}/1850_2014_ini_files/'
        ini_2015_2100 = f'{WORK_DIR}/2015_2100_ini_files/'
        initial_folder = f'{WORK_DIR}/Results/1850_2014/SSP2_{model}'
        results_folder_1850_2014 = initial_folder
        results_folder_2015_2100 = f"{WORK_DIR}/Results/2015_2100/without/SSP5_{model}"

        os.makedirs(results_folder_1850_2014, exist_ok=True)
        os.makedirs(results_folder_2015_2100, exist_ok=True)

        for _, row in networks_df.iterrows():
            network_id = row['Network ID']

            # Run initial simulation
            SpeciesPopEnd, _ = run_initial_simulation(network_id, model, ini_folder, results_folder_1850_2014)

            # Run subsequent simulation for 1850-2014
            run_subsequent_simulation_1850_2014(network_id, model, SpeciesPopEnd, ini_1850_2014, results_folder_1850_2014)

            # Run subsequent simulation for 2015-2100
            run_subsequent_simulation_2015_2100(network_id, model, ini_2015_2100, initial_folder, results_folder_2015_2100)

    print("Simulations completed for all models and scenarios.")

if __name__ == "__main__":
    main()


In [ ]:
#single pollinator management (automated for all ssp, all model)
def run_subsequent_simulation_2015_2100(network_id, model, ssp, ini_folder, initial_folder, results_folder):
    ini_path = os.path.join(ini_folder, f"{model}_{network_id}_{ssp}.ini")
    config = pen.GlobalConfig(ini_path)
    Sampler = pen.SimSpecsSampler(config.params)

    for i in range(len(Sampler.AllSimSpecs)):
        temp_csv = os.path.join(initial_folder,f"SSP2_{model}", f"without_{model}_SSP2_pop_{network_id}_1850_2014{str(Sampler.AllSimSpecs[i]['gamma0']).replace('.', 'p')}.csv")
        SpeciesInitial = pd.read_csv(temp_csv).iloc[-1]
        SpeciesInitial.drop(['Unnamed: 0', 'year', 'month', 'date'], inplace=True)
        
        runner = pen.Simulator(config, Sampler.AllSimSpecs[i])
        results_filename = os.path.join(results_folder, f"single_{model}_{ssp}_pop_{network_id}_{str(runner.sim_specs['gamma0']).replace('.', 'p')}.csv")
        
        _, Data = runner.run(
            IsManaged=None,
            IsTemperatureDependent=True,
            SpeciesPopEnd=SpeciesInitial,
            t_i=None,
            t_f=None,
            t_steps=None,
            show_progress=True,
            show_plot=False,
            ReturnPopEnd=True,
            save_results=True,
            results_filename=results_filename,
            ReturnData=True
        )

# Main function to execute the simulations for all models and SSPs
def main():
    csv_file = f'{WORK_DIR}/Network_location/Network_location.csv'
    networks_df = pd.read_csv(csv_file)

    ini_folder = f'{WORK_DIR}/2015_2100_single_ini_files/'
    initial_folder = f'{WORK_DIR}/Results/1850_2014'
    single_results_2015_2100_base = f"{WORK_DIR}/Results/2015_2100/single"

    models = ['BCC','CESM2','EC','AWI', 'FGOALS', 'MRI', 'INM', 'CMCC', 'NorESM2', 'EC_Veg']
    ssp_scenarios = ['SSP2', 'SSP5']

    for model in models:
        for ssp in ssp_scenarios:
            results_folder = os.path.join(single_results_2015_2100_base, f"{ssp}_{model}")
            os.makedirs(results_folder, exist_ok=True)

            for _, row in networks_df.iterrows():
                network_id = row['Network ID']
                run_subsequent_simulation_2015_2100(network_id, model, ssp, ini_folder, initial_folder, results_folder)

    print("Simulations completed for all networks, models, and SSPs.")

if __name__ == "__main__":
    main()


In [ ]:
#multi pollinator management (automated for all ssp, all model)
def run_subsequent_simulation_2015_2100(network_id, model, ssp, ini_folder, initial_folder, results_folder):
    ini_path = os.path.join(ini_folder, f"{model}_{network_id}_{ssp}.ini")
    config = pen.GlobalConfig(ini_path)
    Sampler = pen.SimSpecsSampler(config.params)

    for i in range(len(Sampler.AllSimSpecs)):
        temp_csv = os.path.join(initial_folder,f"SSP2_{model}", f"without_{model}_SSP2_pop_{network_id}_1850_2014{str(Sampler.AllSimSpecs[i]['gamma0']).replace('.', 'p')}.csv")
        SpeciesInitial = pd.read_csv(temp_csv).iloc[-1]
        SpeciesInitial.drop(['Unnamed: 0', 'year', 'month', 'date'], inplace=True)
        
        runner = pen.Simulator(config, Sampler.AllSimSpecs[i])
        results_filename = os.path.join(results_folder, f"multi_{model}_{ssp}_pop_{network_id}_{str(runner.sim_specs['gamma0']).replace('.', 'p')}.csv")
        
        _, Data = runner.run(
            IsManaged=None,
            IsTemperatureDependent=True,
            SpeciesPopEnd=SpeciesInitial,
            t_i=None,
            t_f=None,
            t_steps=None,
            show_progress=True,
            show_plot=False,
            ReturnPopEnd=True,
            save_results=True,
            results_filename=results_filename,
            ReturnData=True
        )

# Main function to execute the simulations for all models and SSPs
def main():
    csv_file = f'{WORK_DIR}/Network_location/Network_location.csv'
    networks_df = pd.read_csv(csv_file)

    ini_folder = f'{WORK_DIR}/2015_2100_multi_ini_files/'
    initial_folder = f'{WORK_DIR}/Results/1850_2014'
    multi_results_2015_2100_base = f"{WORK_DIR}/Results/2015_2100/multi"

    models = ['BCC','CESM2','EC','AWI', 'FGOALS', 'MRI', 'INM', 'CMCC', 'NorESM2', 'EC_Veg']
    ssp_scenarios = ['SSP2', 'SSP5']

    for model in models:
        for ssp in ssp_scenarios:
            results_folder = os.path.join(multi_results_2015_2100_base, f"{ssp}_{model}")
            os.makedirs(results_folder, exist_ok=True)

            for _, row in networks_df.iterrows():
                network_id = row['Network ID']
                run_subsequent_simulation_2015_2100(network_id, model, ssp, ini_folder, initial_folder, results_folder)

    print("Simulations completed for all networks, models, and SSPs.")

if __name__ == "__main__":
    main()


In [ ]:
#single plant management (automated for all ssp, all model)
def run_subsequent_simulation_2015_2100(network_id, model, ssp, ini_folder, initial_folder, results_folder):
    ini_path = os.path.join(ini_folder, f"{model}_{network_id}_{ssp}.ini")
    config = pen.GlobalConfig(ini_path)
    Sampler = pen.SimSpecsSampler(config.params)

    for i in range(len(Sampler.AllSimSpecs)):
        temp_csv = os.path.join(initial_folder,f"SSP2_{model}", f"without_{model}_SSP2_pop_{network_id}_1850_2014{str(Sampler.AllSimSpecs[i]['gamma0']).replace('.', 'p')}.csv")
        SpeciesInitial = pd.read_csv(temp_csv).iloc[-1]
        SpeciesInitial.drop(['Unnamed: 0', 'year', 'month', 'date'], inplace=True)
        
        runner = pen.Simulator(config, Sampler.AllSimSpecs[i])
        results_filename = os.path.join(results_folder, f"single_{model}_{ssp}_pop_{network_id}_{str(runner.sim_specs['gamma0']).replace('.', 'p')}.csv")
        
        _, Data = runner.run(
            IsManaged=None,
            IsTemperatureDependent=True,
            SpeciesPopEnd=SpeciesInitial,
            t_i=None,
            t_f=None,
            t_steps=None,
            show_progress=True,
            show_plot=False,
            ReturnPopEnd=True,
            save_results=True,
            results_filename=results_filename,
            ReturnData=True
        )

# Main function to execute the simulations for all models and SSPs
def main():
    csv_file = f'{WORK_DIR}/Network_location/Network_location.csv'
    networks_df = pd.read_csv(csv_file)

    ini_folder = f'{WORK_DIR}/2015_2100_plant_single_ini_files/'
    initial_folder = f'{WORK_DIR}/Results/1850_2014'
    single_results_2015_2100_base = f"{WORK_DIR}/Results/2015_2100/single_plant"

    models = ['BCC','CESM2','EC','AWI', 'FGOALS', 'MRI', 'INM', 'CMCC', 'NorESM2', 'EC_Veg']
    ssp_scenarios = ['SSP2', 'SSP5']

    for model in models:
        for ssp in ssp_scenarios:
            results_folder = os.path.join(single_results_2015_2100_base, f"{ssp}_{model}")
            os.makedirs(results_folder, exist_ok=True)

            for _, row in networks_df.iterrows():
                network_id = row['Network ID']
                run_subsequent_simulation_2015_2100(network_id, model, ssp, ini_folder, initial_folder, results_folder)

    print("Simulations completed for all networks, models, and SSPs.")

if __name__ == "__main__":
    main()


In [ ]:
#multi plant management (automated for all ssp, all model)
def run_subsequent_simulation_2015_2100(network_id, model, ssp, ini_folder, initial_folder, results_folder):
    ini_path = os.path.join(ini_folder, f"{model}_{network_id}_{ssp}.ini")
    config = pen.GlobalConfig(ini_path)
    Sampler = pen.SimSpecsSampler(config.params)

    for i in range(len(Sampler.AllSimSpecs)):
        temp_csv = os.path.join(initial_folder,f"SSP2_{model}", f"without_{model}_SSP2_pop_{network_id}_1850_2014{str(Sampler.AllSimSpecs[i]['gamma0']).replace('.', 'p')}.csv")
        SpeciesInitial = pd.read_csv(temp_csv).iloc[-1]
        SpeciesInitial.drop(['Unnamed: 0', 'year', 'month', 'date'], inplace=True)
        
        runner = pen.Simulator(config, Sampler.AllSimSpecs[i])
        results_filename = os.path.join(results_folder, f"multi_{model}_{ssp}_pop_{network_id}_{str(runner.sim_specs['gamma0']).replace('.', 'p')}.csv")
        
        _, Data = runner.run(
            IsManaged=None,
            IsTemperatureDependent=True,
            SpeciesPopEnd=SpeciesInitial,
            t_i=None,
            t_f=None,
            t_steps=None,
            show_progress=True,
            show_plot=False,
            ReturnPopEnd=True,
            save_results=True,
            results_filename=results_filename,
            ReturnData=True
        )

# Main function to execute the simulations for all models and SSPs
def main():
    csv_file = f'{WORK_DIR}/Network_location/Network_location.csv'
    networks_df = pd.read_csv(csv_file)

    ini_folder = f'{WORK_DIR}/2015_2100_plant_multi_ini_files/'
    initial_folder = f'{WORK_DIR}/Results/1850_2014'
    single_results_2015_2100_base = f"{WORK_DIR}/Results/2015_2100/multi_plant"

    models = ['BCC','CESM2','EC','AWI', 'FGOALS', 'MRI', 'INM', 'CMCC', 'NorESM2', 'EC_Veg']
    ssp_scenarios = ['SSP2', 'SSP5']

    for model in models:
        for ssp in ssp_scenarios:
            results_folder = os.path.join(single_results_2015_2100_base, f"{ssp}_{model}")
            os.makedirs(results_folder, exist_ok=True)

            for _, row in networks_df.iterrows():
                network_id = row['Network ID']
                run_subsequent_simulation_2015_2100(network_id, model, ssp, ini_folder, initial_folder, results_folder)

    print("Simulations completed for all networks, models, and SSPs.")

if __name__ == "__main__":
    main()
